In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer

from collections import Counter
import xgboost as xgb 

In [3]:
train_id = pd.read_csv('train_identity.csv')
train_tr = pd.read_csv('train_transaction.csv')
test_id = pd.read_csv('test_identity.csv')
test_tr = pd.read_csv('test_transaction.csv')

In [5]:
train = train_tr.merge(train_id, how='left', on = 'TransactionID')
#test = test_tr.merge(test_id, how='left', left_index=True, right_index=True)
test=test_tr.merge(test_id, how='left',on='TransactionID')

print(train.shape)
print(test.shape)

y_train = train['isFraud'].copy()

# Drop target, fill in NaNs
X_train = train.drop('isFraud', axis=1)
X_test = test.copy()
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

(590540, 434)
(506691, 433)


In [7]:
X_train.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,86400,68.5,W,13926,-999.0,150.0,discover,142.0,credit,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
2,2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
3,2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,-999,-999.0,-999,-999,-999,-999,-999,-999,-999,-999
4,2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [9]:
from sklearn import preprocessing

In [11]:
X_test.columns = X_test.columns.str.replace('-', '_')

In [13]:
for f in X_train.columns:
    # Check if the column exists in both X_train and X_test
    if f in X_test.columns:
        if X_train[f].dtype == 'object' or X_test[f].dtype == 'object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f].values) + list(X_test[f].values))
            X_train[f] = lbl.transform(list(X_train[f].values))
            X_test[f] = lbl.transform(list(X_test[f].values))
    else:
        # Optionally handle columns that exist in X_train but not in X_test
        print(f"Column {f} exists in X_train but not in X_test.")
  

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20)

In [22]:

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score

# Assuming X_train, X_valid, y_train, y_valid are already defined

# Calculate contamination ratio
contamination_ratio = y_train.value_counts()[1] / y_train.shape[0]

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# Initialize Isolation Forest
iso_forest = IsolationForest(
    n_estimators=1000,
    max_samples=0.8,
    contamination=contamination_ratio,
    max_features=0.8,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)

# Fit the model
iso_forest.fit(X_train_scaled)

# Get anomaly scores
anomaly_scores_train = iso_forest.decision_function(X_train_scaled)
anomaly_scores_valid = iso_forest.decision_function(X_valid_scaled)

# Predict anomalies
predictions = iso_forest.predict(X_valid_scaled)
predictions = np.where(predictions == -1, 1, 0)

# Evaluate
print(classification_report(y_valid, predictions))

roc_auc = roc_auc_score(y_valid, -anomaly_scores_valid)
print(f"ROC AUC Score: {roc_auc}")


              precision    recall  f1-score   support

           0       0.97      0.97      0.97    113972
           1       0.10      0.10      0.10      4136

    accuracy                           0.94    118108
   macro avg       0.53      0.53      0.53    118108
weighted avg       0.94      0.94      0.94    118108

ROC AUC Score: 0.7229066102699492


In [24]:
xgb_model = xgb.XGBClassifier(
    max_depth=8,              # Max depth of trees
    learning_rate=0.005,       # Learning rate (step size shrinkage)
    n_estimators=8000,         # Number of trees to be built
    subsample=0.8,            # Fraction of samples used per tree
    colsample_bytree=1,     # Fraction of features used per tree
    colsample_bylevel=0.8,    # Fraction of features per tree level
    min_child_weight=1,       # Minimum sum of instance weight in a child
    gamma=0.005,                # Minimum loss reduction required for split
    scale_pos_weight=1,       # Balancing positive/negative classes
    reg_alpha=0.4,           # L1 regularization
    reg_lambda=0.15,           # L2 regularization
    tree_method='hist',       # Use histogram-based algorithm
    random_state=42,          # Seed for reproducibility
    objective='binary:logistic',  # For binary classification
    eval_metric='auc',        # Evaluation metric
    n_jobs=-1                 # Use all available cores
)

In [26]:
# Combine with XGBoost
X_train_augmented = X_train.copy()
X_valid_augmented = X_valid.copy()
X_train_augmented['anomaly_score'] = anomaly_scores_train
X_valid_augmented['anomaly_score'] = anomaly_scores_valid

# Retrain XGBoost with the new feature
xgb_model.fit(X_train_augmented, y_train)

# Predict and evaluate
xgb_predictions = xgb_model.predict(X_valid_augmented)
print(classification_report(y_valid, xgb_predictions))


              precision    recall  f1-score   support

           0       0.99      1.00      0.99    113972
           1       0.96      0.61      0.74      4136

    accuracy                           0.99    118108
   macro avg       0.97      0.80      0.87    118108
weighted avg       0.98      0.99      0.98    118108



In [28]:
roc_auc_xgb = roc_auc_score(y_valid, xgb_predictions)
print(f"ROC AUC Score: {roc_auc}")

ROC AUC Score: 0.7229066102699492


In [30]:
train_scores = pd.DataFrame({'AnomalyScore': anomaly_scores_train, 'Label': y_train})
print(train_scores.corr())


              AnomalyScore     Label
AnomalyScore      1.000000 -0.136014
Label            -0.136014  1.000000


In [32]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=contamination_ratio, novelty=True)
lof.fit(X_train_scd)
anomaly_scores_lof = -lof.decision_function(X_train_scaled)

In [37]:

# Assuming you have LOF anomaly scores for the training set
lof_scores_train = -lof.negative_outlier_factor_

# Create a DataFrame to compute correlation
lof_scores_df = pd.DataFrame({'LOF_Score': lof_scores_train, 'Label': y_train})
correlation = lof_scores_df.corr()
print(correlation)


           LOF_Score     Label
LOF_Score   1.000000  0.010271
Label       0.010271  1.000000
